In [1]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sub = pd.read_csv("./data/sample_submission.csv")

In [4]:
count_vectorizer = feature_extraction.text.CountVectorizer()
## let's get counts for the first 5 tweets in the data
example_train_vectors =count_vectorizer.fit_transform(train.text.head(5))

In [7]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
## .todense() converts the sparse matrix to a dense matrix
print(example_train_vectors[0].todense().shape)




(1, 54)


In [9]:
#Now let's create vectors for all of our tweets.
train_vectors = count_vectorizer.fit_transform(train.text)
test_vectors = count_vectorizer.transform(test.text)

In [15]:
# model
clf = linear_model.RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [16]:
#cross vaildation with scoring metric as f1
scores = model_selection.cross_val_score(clf, train_vectors, train.target, cv=3, scoring='f1')
print(scores)

/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


[0.59421842 0.56526006 0.64149093]


/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [17]:
# fit the model
clf.fit(train_vectors, train.target)


/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


RidgeClassifier(normalize=False)

In [18]:
# do the prediction 
sub['target'] = clf.predict(test_vectors)

In [19]:
sub.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [ ]:
sub.to_csv('submission.csv', index=False)